## How to add new AL query strategies / unlabeled pool subsampling strategies

This notebook demonstrates three simple steps to benchmark a new query / unlabeled pool subsampling strategy.

#### 1. Prepare the file and the global variable

In [1]:
# Where the file with the strategy is located
FOLDER_WITH_STRATEGIES = "custom_strategy"
!mkdir $FOLDER_WITH_STRATEGIES
# Name of the AL strategy & file
AL_STRATEGY_NAME = "least_confidence.py"
# Name of the unlabeled pool subsampling strategy addition
SUBSAMPLING_STRATEGY_NAME = "top_from_previous_iteration_subsampling.py"
CUR_PATH = !pwd
# Absolute path to the AL strategy
PATH_TO_AL_STRATEGY = f"{CUR_PATH[0]}/{FOLDER_WITH_STRATEGIES}/{AL_STRATEGY_NAME}"
# Absolute path to the strategy
PATH_TO_SUBSAMPLING_STRATEGY = (
    f"{CUR_PATH[0]}/{FOLDER_WITH_STRATEGIES}/{SUBSAMPLING_STRATEGY_NAME}"
)

mkdir: cannot create directory ‘custom_strategy’: File exists


#### 2. Write your strategies

In [2]:
%%writefile $PATH_TO_AL_STRATEGY

import numpy as np

def least_confidence(model, X_pool, n_instances, **kwargs):
    probas = model.predict_proba(X_pool)
    uncertainty_estimates = 1 - probas.max(axis=1)
    query_idx = np.argsort(-uncertainty_estimates)[:n_instances]
    query = X_pool.select(query_idx)
    return query_idx, query, uncertainty_estimates

Overwriting /home/atsvigun/active_learning/examples/custom_strategy/least_confidence.py


In [3]:
%%writefile $PATH_TO_SUBSAMPLING_STRATEGY

import numpy as np

def top_from_previous_iteration_subsampling(uncertainty_estimates, gamma_or_k_confident_to_save, **kwargs):
    if isinstance(gamma_or_k_confident_to_save, float):
        gamma_or_k_confident_to_save = int(
            gamma_or_k_confident_to_save * len(uncertainty_estimates)
        )
    argsort = np.argsort(-uncertainty_estimates)
    return argsort[:gamma_or_k_confident_to_save]

Overwriting /home/atsvigun/active_learning/examples/custom_strategy/top_from_previous_iteration_subsampling.py


#### 3. Use your strategies:

- AL strategy: `config.al.strategy=$PATH_TO_AL_STRATEGY`

- Unlabeled pool subsampling strategy: `config.al.sampling_type=$PATH_TO_SUBSAMPLING_STRATEGY`

Test with 1 GPU: (substitute `custom_strategy/least_confidence` & `custom_strategy/top_from_previous_iteration_subsampling` with your strategies name)

In [4]:
%%bash
CUDA_VISIBLE_DEVICES='0' HYDRA_CONFIG_PATH=../acleto/al_benchmark/configs \
HYDRA_CONFIG_NAME=al_cls python ../scripts/run_active_learning.py \
al.strategy=custom_strategy/least_confidence \
al.sampling_type=custom_strategy/top_from_previous_iteration_subsampling \
acquisition_model.checkpoint=distilbert-base-uncased \
al.num_queries=2

[2022-09-14 14:10:22,670][root][INFO] - Work dir: /home/atsvigun/active_learning/examples/workdir/run_active_learning/2022-09-14/14-10-22_42_roberta_base_custom_strategy_least_confidence
[2022-09-14 14:10:39,668][root][INFO] - Successfully loaded BARTScore and SummaC models.


/home/atsvigun/anaconda3/envs/al/lib/python3.9/site-packages/transformers/image_utils.py:222: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  def resize(self, image, size, resample=PIL.Image.BILINEAR, default_to_square=True, max_size=None):


[2022-09-14 14:10:41,245][root][INFO] - output_dir: ./workdir/run_active_learning
[2022-09-14 14:10:41,245][root][INFO] - seed: 42
[2022-09-14 14:10:41,245][root][INFO] - cuda_device: 0
[2022-09-14 14:10:41,245][root][INFO] - cache_dir: ././workdir/run_active_learning/cache_42_roberta_base
[2022-09-14 14:10:41,245][root][INFO] - cache_model_and_dataset: False
[2022-09-14 14:10:41,246][root][INFO] - framework: transformers
[2022-09-14 14:10:41,246][root][INFO] - task: cls
[2022-09-14 14:10:41,246][root][INFO] - offline_mode: False
[2022-09-14 14:10:41,246][root][INFO] - data
[2022-09-14 14:10:41,246][root][INFO] - 	dataset_name: ag_news
[2022-09-14 14:10:41,246][root][INFO] - 	text_name: text
[2022-09-14 14:10:41,246][root][INFO] - 	label_name: label
[2022-09-14 14:10:41,246][root][INFO] - 	labels_to_remove: None
[2022-09-14 14:10:41,246][root][INFO] - 	path: datasets
[2022-09-14 14:10:41,246][root][INFO] - 	train_size_split: 0.9
[2022-09-14 14:10:41,247][root][INFO] - 	seed: 42
[2022-0

100%|██████████| 2/2 [00:00<00:00, 11.60it/s]


[2022-09-14 14:10:47,664][root][INFO] - Loaded train size: 120000
[2022-09-14 14:10:47,665][root][INFO] - Loaded dev size: 7600
[2022-09-14 14:10:47,665][root][INFO] - Dev dataset coincides with test dataset
[2022-09-14 14:10:48,033][root][INFO] - Seeding dataset size: 1200
[2022-09-14 14:10:48,033][root][INFO] - Pool size: 118800
[2022-09-14 14:10:48,039][root][INFO] - Done.
[2022-09-14 14:10:48,040][root][INFO] - Starting active learning...
[2022-09-14 14:10:48,053][root][INFO] - Constructing the acquisition model...


...

[2022-09-14 14:17:53,845][root][INFO] - Done with the model fit.
[2022-09-14 14:17:53,857][root][INFO] - ############### Evaluating the acquisition model. ###############



100%|██████████| 8/8 [00:00<00:00,  9.35ba/s]


[2022-09-14 14:18:02,011][root][INFO] - AL iteration 1:
Acquisition model:
[2022-09-14 14:18:02,015][root][INFO] - test_loss: 0.5024663209915161
[2022-09-14 14:18:02,017][root][INFO] - test_accuracy: 0.9232894736842105
[2022-09-14 14:18:02,018][root][INFO] - test_f1_micro: 0.9232894736842105
[2022-09-14 14:18:02,018][root][INFO] - test_f1_macro: 0.923027730225176
[2022-09-14 14:18:02,019][root][INFO] - test_f1_weighted: 0.923027730225176
[2022-09-14 14:18:02,019][root][INFO] - test_runtime: 7.0611
[2022-09-14 14:18:02,019][root][INFO] - test_samples_per_second: 1076.32
[2022-09-14 14:18:02,019][root][INFO] - test_steps_per_second: 10.763
[2022-09-14 14:18:02,100][root][INFO] - =================AL iteration #2 started.=================


AL queries done:   7%|▋         | 1/15 [05:14<1:13:21, 314.36s/it]
Traceback (most recent call last):
  File "/home/atsvigun/active_learning/examples/../scripts/run_active_learning.py", line 137, in <module>
    main()
  File "/home/atsvigun/anaconda3/envs/al/lib/python3.9/site-packages/hydra/main.py", line 49, in decorated_main
    _run_hydra(
  File "/home/atsvigun/anaconda3/envs/al/lib/python3.9/site-packages/hydra/_internal/utils.py", line 367, in _run_hydra
    run_and_report(
  File "/home/atsvigun/anaconda3/envs/al/lib/python3.9/site-packages/hydra/_internal/utils.py", line 211, in run_and_report
    return func()
  File "/home/atsvigun/anaconda3/envs/al/lib/python3.9/site-packages/hydra/_internal/utils.py", line 368, in <lambda>
    lambda: hydra.run(
  File "/home/atsvigun/anaconda3/envs/al/lib/python3.9/site-packages/hydra/_internal/hydra.py", line 97, in run
    ret = run_job(
  File "/home/atsvigun/anaconda3/envs/al/lib/python3.9/site-packages/hydra/core/utils.py", line 160

Error while terminating subprocess (pid=14253): 


The results will be located in the file `workdir/run_active_learning/TODAY_DATE/TIME_SEED_MODEL/acquisition_metrics.json`.